## Reexecução

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.environ['SNORKELDB'] = 'postgres:///lzirondi'
from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
relation = candidate_subclass('Causa', ['Substância', 'Evento_Adverso'])

## Gerando o Dicionário de Descrisções

In [ ]:
import json

with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/dic/LIWC2015_pt_desc_only.dic', 'r', encoding='utf-8-sig') as liwc_desc:
    desc_txt = liwc_desc.readlines()

desc_dic = dict()

for line in desc_txt:
    line = line.split('\t')
    
    cat = line[1].replace('\n', '')
    cat = cat.split(' ')[1:]
    aux = ' '
    cat = ''.join(cat)
    
    cat = cat.replace('(', '')
    cat = cat.replace(')', '')
    
    desc_dic[line[0]] = cat
    
with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/LIWC2015_pt_desc_only.json', 'w', encoding='utf-8') as json_file:
    json.dump(desc_dic, json_file, indent=0)

## Gerado o Dicionário das Palavras do LIWC

In [ ]:
import json

with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/dic/LIWC2015_pt_no_desc.dic', 'r', encoding='utf-8-sig') as liwc_words:
    words_txt = liwc_words.readlines()

words_dic = dict()

for line in words_txt:
    tags = []
    line = line.split('\t')
    for element in line:
        if element != '' and element != '\n':
            tags.append(element)
    
    words_dic[tags[0]] = tags[1:]
    
with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/LIWC2015_pt_no_desc.json', 'w', encoding='utf-8') as json_file:
    json.dump(words_dic, json_file, indent=2)

## Juntado os dois dicionários (criando um dict do LIWC com as descrições ao invés dos números) ?????

In [ ]:
import copy
import json

with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/LIWC2015_pt_desc_only.json', 'r', encoding='utf-8') as desc_file:
    desc_dict = json.load(desc_file)
#print(desc_dict)    
with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/LIWC2015_pt_no_desc.json', 'r', encoding='utf-8') as words_file:
    words_dict = json.load(words_file)
#print(words_dict)
for key in words_dict:
    tags = words_dict[key]
    
    for i in range(len(tags)):
        #print(tags[i].replace('/n', ''))
        tags[i] = desc_dict[tags[i].replace('\n', '')]
    
    words_dict[key] = tags

with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/LIWC2015_pt.json', 'w', encoding='utf-8') as json_file:
    json.dump(words_dict, json_file, indent=2)

In [ ]:
'''
candidatos interessantes:

split 0 - 100
    Alergia
split 0 - 200
    Falta de pontuação prejudica o entendimento do tweet
split 0 - 333-332
    Exemplo simples
split 0 - 838-853
    Vários candidatos numa mesma sentença
'''
candidate = session.query(relation).filter(relation.split == 0).all()[332]
print_candidate(candidate)
print(get_stable_id(candidate))
print(alt_get_stable_id(candidate))


print(dir(candidate.get_contexts()[0]))
print(candidate.get_contexts()[0].sentence_id)

## Aux functions

In [2]:
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text, get_text_splits,
    is_inverted,
)

def get_text(cand):
    return cand.get_parent()._asdict().get('text')

def get_spans(cand):
    return [cand.get_contexts()[0].get_span(), cand.get_contexts()[1].get_span()]

def get_spans_complex(cand):
    if(is_inverted(cand)):
        return {0:cand.get_contexts()[1].get_span(), 1:cand.get_contexts()[0].get_span()}
    else:
        return {0:cand.get_contexts()[0].get_span(), 1:cand.get_contexts()[1].get_span()}

def get_stable_id(cand):
    return cand[0].stable_id.split(':')[0]

def alt_get_stable_id(cand):
    return cand[1].stable_id.split(':')[0]

def highlight_spans(word_list, span_list):
    s = ''
    for word in word_list:
        if word in span_list:
            s += '*' + word + '* '
        else:
            s += word + ' '
    return s

def print_candidate(cand, switch=False):
    if switch: print("Candidato: " + str(cand), end='\n')
    print("Tweet: " + highlight_spans(get_text(cand).split(' '), get_spans(cand)))

In [3]:
import json
#test stuff
#ids = [100,200,332,333,838,839,840,853]

candidates = (session.query(relation).filter(relation.split == 0).all() + 
    session.query(relation).filter(relation.split == 1).all() + 
    session.query(relation).filter(relation.split == 2).all())
#candidates = []
#for num in ids:
#    candidates.append(session.query(relation).filter(relation.split == 0).all()[num])

#for ca in candidates:
#    print_candidate(ca)
#/test stuff
############################################################################################
#Vars
from nltk.corpus import stopwords
sw = set(stopwords.words("portuguese"))




#sets before_spans, between_spans and after_spans
before_spans = []
before_word_count = dict()
#before_words
#before_words_liwc
    
between_spans = []
between_word_count = dict()
#between_words
#between_words_liwc

after_spans = []
after_word_count = dict()
#after_words
#after_words_liwc

counts = [before_word_count, between_word_count, after_word_count]

checked_ids = set()
#############################################################################################


for i in range(len(candidates)):
#############################################################################################
#Checking if the id was already processed
    stable_id = get_stable_id(candidates[i])
    #print(stable_id)
    if stable_id not in checked_ids:
        checked_ids.add(stable_id)
        jump = 0
        
        while i+jump < len(candidates):
            if stable_id == get_stable_id(candidates[i+jump]):
                jump += 1
            else: break
        
        candidate_list = []
        if jump > 1:
            for j in range(i, i+jump):
                candidate_list.append(candidates[j])
        else:
            candidate_list.append(candidates[i])
############################################################################################
#Getting the candidates and tweet from the ID being checked
        spans = []
        
        for c in candidate_list:
            spans.append(get_spans_complex(c))
        
        tweet = get_text(candidate_list[0]).replace('\n', '').split(' ')
############################################################################################         
#Removing the spans from the tweet and separating the sentence in before_spans, between_spans and after_spans
        for dic in spans:
            for key in dic:
                for i in range(len(tweet)):
                    if tweet[i] == dic[key]:
                        tweet[i] = key
        
        last_zero = 0
        first_one = 0
        aux = True
        for i in range(len(tweet)):
            if type(tweet[i]) == int:
                if tweet[i] == 0:
                    last_zero = i
                elif aux:
                    first_one = i
                    aux = False
                       
            
        if first_one < last_zero:
            first_one, last_zero = last_zero, first_one
        
        before_spans = tweet[:last_zero]
        between_spans = tweet[last_zero+1:first_one]
        after_spans = tweet[first_one+1:]
                
############################################################################################
        categories_tweet = [before_spans, between_spans, after_spans]
        for categories in categories_tweet:
            if categories == before_spans:
                aux = before_word_count
            elif categories == between_spans:
                aux = between_word_count
            else:
                aux = after_word_count
            
            for word in categories:
                if type(word) != int and word in aux and word not in sw:
                    aux[word] += 1
                elif type(word) != int and word not in sw:
                    aux[word] = 1

############################################################################################
with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/before_word_count.json', 'w', encoding='utf-8') as json_file:
    json.dump(before_word_count, json_file, indent=2)

with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/between_word_count.json', 'w', encoding='utf-8') as json_file:
    json.dump(between_word_count, json_file, indent=2)

with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/after_word_count.json', 'w', encoding='utf-8') as json_file:
    json.dump(after_word_count, json_file, indent=2)
############################################################################################        
print('________________________________')

________________________________


In [20]:
#Test stuff
import json

with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/before_word_count.json', 'r', encoding='utf-8') as json_file:
    dic = json.load(json_file)

import itertools


dic = dict(itertools.islice(dic.items(), 50))
#/test stuff

import json
import re

with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/LIWC2015_pt_no_desc.json', 'r', encoding='utf-8') as json_file:
    liwc = json.load(json_file)


liwc_keys = list(liwc)    
#removing emoticons from liwc
for i in range(25):
    del liwc[liwc_keys[i]]

liwc_keys = list(liwc) 
on_liwc = dict()
not_on_liwc = dict()

a = set()




for key in dic:
    word = key
    aux_bool = True
    
    for k in liwc_keys:
        if len(k) < 3: 
            a.add(k)
            pass
        else:
            regex = re.compile(k)
            if regex.match(word):
                #print(word, k)
                aux_bool = False
                on_liwc[k] = [dic[key], liwc[k]]
                break
    
    if aux_bool:
        not_on_liwc[key] = [dic[key], None]

#print(len(a))
#print(a)
print(on_liwc)
print(len(on_liwc))
print(not_on_liwc)
print(len(not_on_liwc))

{'tom': [300, ['60', '62']], 'café*': [36, ['70', '74', '111']], 'forte': [12, ['1', '13', '21', '30', '31', '80', '83', '100', '102']], 'aparece': [2, ['20', '50', '54', '55', '60', '61', '91']], 'deu': [22, ['20', '40', '90']], 'mãe*': [40, ['40', '41', '43']], 'toda': [23, ['1', '2', '9', '40']], 'hm*': [18, ['120', '122', '124']], 'per': [2, ['1', '11']], 'senti': [12, ['20', '50', '51', '60', '63', '90']], 'acha': [2, ['20', '50', '51', '91']], 'cd*': [11, ['111']], 'pior': [28, ['1', '13', '21', '22', '30', '32', '80', '85']], 'podia': [6, ['1', '12', '20', '50', '54', '90']], 'tortur*': [1, ['30', '32', '34']], 'ser': [59, ['1', '12']], 'amante*': [1, ['40', '42', '70', '73', '80', '81']], 'pesquisa': [5, ['60', '61', '110']], 'dj*': [1, ['60', '62', '111']], 'ano': [23, ['100', '103']], 'risperidona': [1, ['70', '72']], 'mt*': [24, ['120', '122']], 'par': [1, ['25']], 'pública*': [1, ['40']], 'sob': [19, ['1', '11', '80', '83', '100', '102']], 'passa': [21, ['20', '91', '100', 

In [14]:
#Test stuff
import json

with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/before_word_count.json', 'r', encoding='utf-8') as json_file:
    dic = json.load(json_file)

import itertools


dic = dict(itertools.islice(dic.items(), 50))
#/test stuff

import json
import re

with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/LIWC2015_pt_no_desc.json', 'r', encoding='utf-8') as json_file:
    liwc = json.load(json_file)


liwc_keys = list(liwc)    
#removing emoticons from liwc
for i in range(25):
    del liwc[liwc_keys[i]]

liwc_keys = list(liwc) 
on_liwc = dict()
not_on_liwc = dict()

a = set()




for key in dic:
    word = key
    aux_bool = True
    
    for k in liwc_keys:
        if '*' in k:
            r = '^' + k.replace('*','')
        else:
            r = '^' + k
        #print(r)
        regex = re.compile(r)
        if regex.match(word):
            aux_bool = False
            on_liwc[k] = [dic[key], liwc[k]]
            break
    
    if aux_bool:
        not_on_liwc[key] = [dic[key], None]

#print(len(a))
#print(a)
print(on_liwc)
print(len(on_liwc))
print(not_on_liwc)
print(len(not_on_liwc))

{'t+': [194, ['120', '122']], 'ca': [1, ['1', '14']], 'forte': [12, ['1', '13', '21', '30', '31', '80', '83', '100', '102']], 'a': [1, ['1', '2', '3', '7', '9', '10', '11']], 'o': [3, ['1', '2', '3', '7', '9', '10']], 'lo': [1, ['1', '3', '2', '7']], 'de': [3, ['1', '11']], 'mãe*': [40, ['40', '41', '43']], 'hora': [18, ['100', '103']], 'per': [2, ['1', '11']], 'se': [55, ['1', '2', '3', '7', '9', '14', '23', '50', '53', '54', '56']], 'pior': [28, ['1', '13', '21', '22', '30', '32', '80', '85']], 'po': [6, ['120', '122', '125']], 'na': [2, ['1', '10', '11']], 'com': [11, ['1', '11']], 'pesquisa': [5, ['60', '61', '110']], 'do': [1, ['1', '10', '11']], 'risperidona': [1, ['70', '72']], 'mt*': [24, ['120', '122']], 'par': [3, ['25']], 'consulta': [2, ['50', '51']], 'pública*': [1, ['40']], 'sob': [19, ['1', '11', '80', '83', '100', '102']], 'passa': [22, ['20', '91', '100', '101']], 'vou': [135, ['1', '12', '20', '92', '100', '101']], 'pq': [68, ['50', '52', '120', '122']], 'legal': [5, 

In [ ]:
import json

from operator import itemgetter

#for dic in count:
#    aux_dic = []
#    for key in dic:
#        aux.append([key,quant[key]])
   
#    aux_dic = sorted(aux_dic, key=itemgetter(1,0))

#aux_dic = []
#for key in counts[0]:
    aux_dic.append([key, counts[0][key]])

#aux_dic = sorted(aux_dic, key=itemgetter(1,0))

#with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/before_word_count_test.json', 'w', encoding='utf-8') as json_file:
#    json.dump(aux_dic, json_file, indent=2)

with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/before_word_count.json', 'w', encoding='utf-8') as json_file:
    json.dump(before_word_count, json_file, indent=2)

with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/between_word_count.json', 'w', encoding='utf-8') as json_file:
    json.dump(between_word_count, json_file, indent=2)

with open('/home/lzirondi/Github/snorkel/Scripts/LIWC/json/after_word_count.json', 'w', encoding='utf-8') as json_file:
    json.dump(after_word_count, json_file, indent=2)

## _________________________________________________

In [ ]:
%%time

from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

from nltk.corpus import stopwords

cands = [session.query(relation).filter(relation.split == 0).all(), session.query(relation).filter(relation.split == 1).all(), session.query(relation).filter(relation.split == 2).all()]

sw = set(stopwords.words("portuguese"))

quant = dict()

for group in cands:
    for candidate in group:
        for word in get_between_tokens(candidate):
            if word not in sw:
                if word in quant:
                    quant[word] = quant[word] + 1
                else:
                    quant[word] = 1
    

words = []

for key in quant:
    aux = [key,quant[key]]
    words.append(aux)
    
from operator import itemgetter

words = sorted(words, key=itemgetter(1,0))


## NotOLD problema das palavras repetidas

In [ ]:
cands = []
for i in range(838,853):
    cands.append(session.query(relation).filter(relation.split == 0).all()[i])

text = []
for candi in cands:
    text.append(get_text_splits(candi))
#print(len(text)) 15
#print(len(text[1])) 5


print_candidate(cands[0])
for i in range(0, len(text[0])):
    print(i)
    for j in range(0,len(text)):
        print(text[j][i])


## Análise

In [ ]:
#%%time

from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

from nltk.corpus import stopwords

cands = session.query(relation).filter(relation.split == 0).all()

sw = set(stopwords.words("portuguese"))

dicios = [causas, verbos, causas_e_verbos]

#chegar no candidato e ver se entre os tokens tem algo que a gente quer

for candidate in cands:
    for word in get_between_tokens(candidate):
        if word not in sw:
            print(word)
            aux = False
            if causas.get(word):
                causas[word] = causas[word] + 1
                print(word)
                if verbos.get(word):
                    aux = True
                    verbos[word] = verbos[word] + 1
                    causas_e_verbos[word] = causas_e_verbos[word] + 1

            if(aux):
                print(word)
                verbos[word] = verbos[word] + 1

In [ ]:
#%%time

from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

from nltk.corpus import stopwords


import re

numero = re.compile('\D+')
repeticao = re.compile('\w{2,}')

cands = session.query(relation).filter(relation.split == 0).all()

sw = set(stopwords.words("portuguese"))

dic = {}

#nao repetir frases
#ignorar palavras de 1 letra 

for candidate in cands:
    for word in get_between_tokens(candidate):
        if word not in sw and len(word)>1 and numero.match(word):
            if dic.get(word):
                dic[word] = dic[word] + 1
            else:
                dic[word] = 1
                
ordenada = []
maior = ['', 0]

dici = dic


for i in range(0, len(dici)):
    for key in dici:
        if dici[key] > maior[1]:
            maior[0] = key
            maior[1] = dici[key]
    ordenada.append(maior)
    dici.pop(maior[0])
    maior = ['', 0]
                

f = open('/home/lzirondi/Github/snorkel/Scripts/Análise de Palavras/between_tokens_stopFiltered.tsv', 'w', encoding='UTF-8')

for word in ordenada:
    f.write(word[0] + '\t' + str(word[1]) + '\n')
    
f.close()


#sorted_list = []
#for key in copy_dic:
#    sorted_list.append([key,copy_dic[key]])

#from operator import itemgetter
#sorted_list = sorted(sorted_list, key=itemgetter(1,0))

#print(sorted_list)


## Testes

In [ ]:
from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

print(len(cands))

rep = []

for candidate in cands:
    s = candidate.get_parent().stable_id.split(':')[0]
    
    if s not in rep:
        rep.append(candidate.id)

print(len(rep))
print(len(cands))
#print(dir(cands[0]))
#print(cands[0].get_parent())

print(cands[0].get_parent().stable_id)


In [15]:
for i in range(10):
    for j in range(20):
        if j == 10:
            break
        else: print(j)

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
